In [1]:
import os
import numpy as np
from PIL import Image
from pathlib import Path
def compute_mean_slices(input_dir, output_dir, group_size=10):
    os.makedirs(output_dir, exist_ok=True)

    # Get and sort all slice file names
    slices = sorted(f for f in os.listdir(input_dir) if f.endswith('.jpg'))

    for i in range(0, len(slices), group_size):
        group = slices[i:i + group_size]
        imgs = [np.array(Image.open(os.path.join(input_dir, f)), dtype=np.float32) for f in group]
        mean_img = np.mean(imgs, axis=0).astype(np.uint8)

        out_name = f'mean_{i//group_size + 1:04d}.jpg'
        Image.fromarray(mean_img).save(os.path.join(output_dir, out_name))

In [ ]:
from pathlib import Path
import os

#dir_image = '../data/pictures_process/train_process/'
dir_image = '../data/pictures_raw/train/'
dir_mean_par = '../data/pictures_process/train_process'
parent_dir = Path(dir_image)
subdirs = [p for p in parent_dir.iterdir() if p.is_dir()]

dir_mean = Path(dir_mean_par) / 'tomo_mean'
os.makedirs(dir_mean, exist_ok=True)

for dirr in subdirs:
    tomo_id = dirr.name
    output_dir = dir_mean / tomo_id
    os.makedirs(output_dir, exist_ok=True)
    compute_mean_slices(str(dirr), str(output_dir), group_size=10)

In [ ]:
### count how many tomo_ids have 300 slices

root_folder = '../data/pictures_raw/train'
target_count = 300
valid_tomos = []

# Loop through each tomo_id directory
for tomo_dir_name in os.listdir(root_folder):
    tomo_dir = os.path.join(root_folder, tomo_dir_name)

    if not os.path.isdir(tomo_dir):
        continue

    # Count only .jpg files
    jpg_count = len([
        f for f in os.listdir(tomo_dir)
        if f.lower().endswith('.jpg')
    ])

    if jpg_count == target_count:
        valid_tomos.append(tomo_dir_name)

len(valid_tomos)

In [ ]:
from pathlib import Path
from PIL import Image
import numpy as np
import skimage.io as img
from skimage.exposure import equalize_adapthist
import os

# Paths and parameters
train_folder = Path('../data/pictures_process/train_process')
root_folder = train_folder / 'tomo_mean'
clip_sizes = [1]  # clip limits

#for tomo_dir_name in os.listdir(root_folder):
for tomo_dir_name in valid_tomos:
    tomo_dir = root_folder / tomo_dir_name

    if not tomo_dir.is_dir():
        continue

    for mean_picture_name in os.listdir(tomo_dir):
        mean_picture_path = tomo_dir / mean_picture_name

        # Skip non-images
        if not mean_picture_name.lower().endswith('.jpg'):
            continue

        print(f"Reading: {mean_picture_path}")
        tomo_mean = img.imread(mean_picture_path)

        for clip_size in clip_sizes:
            # Prepare paths
            safe_clip = str(clip_size).replace('.', '')
            output_root = train_folder / f'tomo_adaptequal_{safe_clip}'
            output_dir = output_root / tomo_dir_name

            output_dir.mkdir(parents=True, exist_ok=True)

            # Apply contrast adaptation
            adapt_equalized_picture = equalize_adapthist(tomo_mean, clip_limit=clip_size)
            adapt_equalized_uint8 = (adapt_equalized_picture * 255).astype(np.uint8)

            # Save image with original filename
            output_path = output_dir / mean_picture_name
            im = Image.fromarray(adapt_equalized_uint8)
            im.save(output_path)

            print(f"Saved: {output_path}")